In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import random
import os
import csv
import cv2
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [2]:
# with open('imagesDataset.csv') as csvfile:
#     pathsArray=[]
#     labelsArray=[]
#     readCSV = csv.reader(csvfile, delimiter=',')
#     for row in readCSV:
#         pathsArray.append(row[0])
#         labelsArray.append(row[1])
# print (pathsArray)
#################################
# image = load_img(pathsArray[10])
# plt.imshow(image)
#################################
# def image_processing(image_path):
#     img = cv2.imread(image_path,0)
#     image = np.asarray(bytearray(img.read()), dtype="uint8")
#     image_bgr = cv2.imdecode(image, cv2.IMREAD_COLOR)
#     image_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
#     image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
#     mask = cv2.inRange(image_hsv, (0,255,255), (0,255,255))
# image_processing('CelebA_Spoof\\Data\\train\\98\\live\\286606.jpg')

In [3]:
img_width, img_height = 150, 150

train_data_dir = 'CelebA_Spoof/Data/train'
validation_data_dir = 'CelebA_Spoof/Data/test'
nb_train_samples = 1052
nb_validation_samples = 200
epochs = 10
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 1052 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
65/65 [==============================] - 60s 924ms/step - loss: 0.6071 - accuracy: 0.7307 - val_loss: 0.5739 - val_accuracy: 0.6823
Epoch 2/10
65/65 [==============================] - 51s 785ms/step - loss: 0.3433 - accuracy: 0.8523 - val_loss: 0.6951 - val_accuracy: 0.6042
Epoch 3/10
65/65 [==============================] - 53s 820ms/step - loss: 0.2671 - accuracy: 0.8697 - val_loss: 0.8051 - val_accuracy: 0.6823
Epoch 4/10
65/65 [==============================] - 50s 770ms/step - loss: 0.2349 - accuracy: 0.9102 - val_loss: 0.4754 - val_accuracy: 0.7656
Epoch 5/10
65/65 [==============================] - 51s 779ms/step - loss: 0.2157 - accuracy: 0.9102 - val_loss: 0.4283 - val_accuracy: 0.8229
Epoch 6/10
65/65 [==============================] - 54s 836ms/step - loss: 0.2125 - accuracy: 0.9151 - val_loss: 0.3253 - val_